# Play the Game
Write your first game-playing agent.

<iframe src="https://www.kaggle.com/embed/alexisbcook/play-the-game?cellIds=1&kernelSessionId=126574698" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="Play the Game"></iframe>

### 1) A smarter agent

We can improve the performance without devising a complicated strategy, simply by selecting a winning move, if one is available.

We will create an agent that:
- selects the winning move, if it is available.  (_If there is more than one move that lets the agent win the game, the agent can select any of them._)
- Otherwise, it should select a random move.


In [1]:
import numpy as np

# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid

# Returns True if dropping piece in column results in game win
def check_winning_move(obs, config, col, piece):
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    next_grid = drop_piece(grid, col, piece, config)
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[row,col:col+config.inarow])
            if window.count(piece) == config.inarow:
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(next_grid[row:row+config.inarow,col])
            if window.count(piece) == config.inarow:
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    return False

The `check_winning_move()` function takes four required arguments: 
- `col` is any valid move 
- `piece` is either the agent's mark or the mark of its opponent. 

The function returns `True` if dropping the piece in the provided column wins the game (for either the agent or its opponent), and otherwise returns `False`.  

In [2]:
import random
def agent_q1(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark):
            return col
    return random.choice(valid_moves)

### 2) An even smarter agent

In the previous question, we created an agent that selects winning moves.  In this problem, we'll amend the code to create an agent that can also block its opponent from winning.  In particular, our agent should:
- Select a winning move, if one is available.
- Otherwise, it selects a move to block the opponent from winning, if the opponent has a move that it can play in its next turn to win the game. 
- If neither the agent nor the opponent can win in their next moves, the agent selects a random move.

In [3]:
def agent_q2(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark):
            return col
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark%2+1):
            return col
    return random.choice(valid_moves)

### 3) Looking ahead

So far, we have encoded an agent that always selects the winning move, if it's available.  And, it can also block the opponent from winning.

we might expect that this agent should perform quite well!  But how is it still possible that it can still lose the game?

**Solution**: The agent can still lose the game, if

- the opponent has set up the board so that it can win in the next move by dropping a disc in any of 2 or more columns, or
- the only move that is available to the agent is one where, once played, the opponent can win in the next move.